In [1]:
def path_transacional():
    import os
    if os.path.isfile(r'I:\Cloud\SQLite\10 - Source\transacional\relatorio.csv'):#home
        dir_reds  = r'I:\Cloud\SQLite\10 - Source\transacional\relatorio.csv'
    elif os.path.isfile(r'D:\Cloud\SQLite\10 - Source\transacional\relatorio.csv'):#office
        dir_reds  = r"D:\Cloud\SQLite\10 - Source\transacional\relatorio.csv"
    else:
        dir_reds  = "Arquivo não encontrado!"
    return dir_reds    

path_transacional()

'D:\\Cloud\\SQLite\\10 - Source\\transacional\\relatorio.csv'

In [2]:
import pandas as pd
import numpy as np

df_reds = pd.read_csv(path_transacional(), 
                      delimiter = ';',
                      #engine='python',
                      encoding='cp1252',
                      skiprows = 0)

In [3]:
#transforma os cabeçalhos  em caixa alta e  remove os acentos
def columns():
    for df in [df_reds]:
        df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') #remove acento dos cabeçalhos
        df.columns = df.columns.str.replace(' ', '_')
        df.columns = df.columns.str.upper()
        #df.set_index("NUMERO_REDS", inplace = True)
        
columns()
df_reds.rename(columns = {'NUMERO_DO_REDS': 'NUMERO_REDS', 'Tipo de Relatório': 'RELATORIO'}, inplace = True)
df_reds.columns

Index(['NUMERO_REDS', 'TIPO_DE_RELATORIO', 'DATA/HORA_DE_CRIACAO_DO_REGISTRO',
       'DATA/HORA_DO_FATO', 'NATUREZA_PRINCIPAL', 'ENDERECO_DO_FATO',
       'NUMERO_DO_BO'],
      dtype='object')

In [4]:
df_reds['LOGRADOURO_OCORRENCIA'] = df_reds['ENDERECO_DO_FATO'].apply(lambda x: x.replace("RUA " , "")).str.split("/").str.get(0).str.strip()

df_reds['LOGRADOURO_OCORRENCIA'] = df_reds['LOGRADOURO_OCORRENCIA'].apply(lambda x: x.replace("PRACA " , ""))
df_reds['LOGRADOURO_OCORRENCIA'] = df_reds['LOGRADOURO_OCORRENCIA'].apply(lambda x: x.replace("AVENIDA " , ""))

df_reds['LOGRADOURO_OCORRENCIA'] = df_reds['LOGRADOURO_OCORRENCIA'].apply(lambda x: x.replace("BAIRRO " , ""))

df_reds['LOGRADOURO_OCORRENCIA'] = df_reds['LOGRADOURO_OCORRENCIA'].apply(lambda x: x.replace("POVOADO " , ""))

In [5]:
df_end = df_reds['ENDERECO_DO_FATO'].str.split("-", n=20, expand=True)
    
df_reds["BAIRRO"] = df_end[1].str.strip()
df_reds["MUNICIPIO"] = df_end[2].str.strip()

In [6]:
df_reds['DATA/HORA_DO_FATO'] = pd.to_datetime(df_reds['DATA/HORA_DO_FATO'], format='%d/%m/%Y %H:%M', errors='coerce')

df_reds['DATA_ULTIMA_ATUALIZACAO'] = pd.to_datetime(df_reds['DATA/HORA_DE_CRIACAO_DO_REGISTRO'], format='%d/%m/%Y %H:%M', errors='coerce').dt.date
df_reds['DATA_ULTIMA_ATUALIZACAO'] = pd.to_datetime(df_reds['DATA_ULTIMA_ATUALIZACAO'] , format='%Y/%m/%d', errors='coerce')


df_reds['DATA_FATO'] = df_reds['DATA/HORA_DO_FATO'].dt.date
df_reds['DATA_FATO'] = pd.to_datetime(df_reds['DATA_FATO'], format='%Y/%m/%d', errors='coerce')


df_reds['HORARIO_FATO'] = df_reds['DATA/HORA_DO_FATO'].dt.time


df_reds['TENTADO_CONSUMADO_NAT_PRINCIPAL'] = 'CONSUMADO'

df_reds['ALVO'] = 'OUTROS / IGNORADO'
df_reds['CAUSA_PRESUMIDA'] = 'IGNORADO'
df_reds['FRACAO'] = 'REDS'
df_reds['LATITUDE']  = 0.0
df_reds['LONGITUDE'] = 0.0

In [7]:
def path_classif():
    import os
    if os.path.isfile(r'I:\Cloud\SQLite\10 - Source\validadores\tbl_classificadores_geo.xlsx'):#home
        dir_classif  = r"I:\Cloud\SQLite\10 - Source\validadores\tbl_classificadores_geo.xlsx"
    else:
        os.path.isfile(r'D:\Cloud\SQLite\10 - Source\validadores\tbl_classificadores_geo.xlsx')#office
        dir_classif  = r"D:\Cloud\SQLite\10 - Source\validadores\tbl_classificadores_geo.xlsx"
    return dir_classif      

path_classif()

'D:\\Cloud\\SQLite\\10 - Source\\validadores\\tbl_classificadores_geo.xlsx'

In [8]:
import pandas as pd
import numpy as np
import geopandas as gpd


df_classif  = pd.read_excel(path_classif(), sheet_name='tbl_classificadores')
df_classif[['LATITUDE', 'LONGITUDE']] = df_classif['COORDENADAS'].str.split(',', 1, expand=True)
df_classif['LATITUDE'] = df_classif['LATITUDE'].astype(np.float64)
df_classif['LONGITUDE'] = df_classif['LONGITUDE'].astype(np.float64)

In [9]:
df_classif['VALIDADOR'] = df_classif['VALIDADOR'].str.upper().fillna("")
df_classif.drop_duplicates(['MUNICIPIO', 'VALIDADOR_TIPO', 'VALIDADOR'], inplace = True)
df_classif.set_index(df_classif['MUNICIPIO'] + " " + df_classif['VALIDADOR_TIPO'] + " " + df_classif['VALIDADOR'], inplace=True, verify_integrity=True)

In [ ]:
df_reds

In [10]:
def preenche_latitude(row):
#------------------------------------------------INICIO
    municipio = row['MUNICIPIO']
    latitude  = row['LATITUDE']
#------------------------------------------------INICIO 
    if latitude != 0.0:        
        return latitude
#----------------------------------------------INICIO CLASSIFICAÇÃO POR CRITERIOS COMPLEMENTARES
    elif ( municipio + ' N_REDS ' + row['NUMERO_REDS'] ) in df_classif.index:
        return df_classif.loc[municipio + ' N_REDS ' + row['NUMERO_REDS'], 'LATITUDE'] 
    elif ( municipio + ' BAIRRO ' + row['BAIRRO'].upper() ) in df_classif.index:
        return df_classif.loc[municipio + ' BAIRRO ' + row['BAIRRO'], 'LATITUDE']
    
    elif ( municipio + ' LOGRADOURO ' + row['LOGRADOURO_OCORRENCIA'] ) in df_classif.index:
        return df_classif.loc[municipio + ' LOGRADOURO ' + row['LOGRADOURO_OCORRENCIA'], 'LATITUDE']
    
    
    elif ( municipio + ' CIDADE ' + row['MUNICIPIO'] ) in df_classif.index:
        return df_classif.loc[municipio + ' CIDADE ' + row['MUNICIPIO'], 'LATITUDE']    

    
    else:
        return '00.000'
def preenche_longitude(row):
#------------------------------------------------INICIO
    municipio = row['MUNICIPIO']
    longitude = row['LONGITUDE']
#------------------------------------------------INICIO 
    if longitude != 0.0:        
        return longitude
#----------------------------------------------INICIO CLASSIFICAÇÃO POR CRITERIOS COMPLEMENTARES
    elif ( municipio + ' N_REDS ' + row['NUMERO_REDS'] ) in df_classif.index:
        return df_classif.loc[municipio + ' N_REDS ' + row['NUMERO_REDS'], 'LONGITUDE'] 
    elif ( municipio + ' BAIRRO ' + row['BAIRRO'].upper() ) in df_classif.index:
        return df_classif.loc[municipio + ' BAIRRO ' + row['BAIRRO'], 'LONGITUDE']
    
    elif ( municipio + ' LOGRADOURO ' + row['LOGRADOURO_OCORRENCIA'] ) in df_classif.index:
        return df_classif.loc[municipio + ' LOGRADOURO ' + row['LOGRADOURO_OCORRENCIA'], 'LONGITUDE']
       
    elif ( municipio + ' CIDADE ' + row['MUNICIPIO'] ) in df_classif.index:
        return df_classif.loc[municipio + ' CIDADE ' + row['MUNICIPIO'], 'LONGITUDE']    

    else:
        return '00.000'
    
    
def preenche_valid(row):
#------------------------------------------------INICIO
    municipio = row['MUNICIPIO']
    latitude  = row['LATITUDE']
#------------------------------------------------INICIO 
    if latitude != 0.0:        
        return "REDS"
#----------------------------------------------INICIO CLASSIFICAÇÃO POR CRITERIOS COMPLEMENTARES
    elif ( municipio + ' N_REDS ' + row['NUMERO_REDS'] ) in df_classif.index:
        return df_classif.loc[municipio + ' N_REDS ' + row['NUMERO_REDS'], 'VALIDADOR_TIPO'] 
    elif ( municipio + ' BAIRRO ' + row['BAIRRO'].upper() ) in df_classif.index:
        return df_classif.loc[municipio + ' BAIRRO ' + row['BAIRRO'], 'VALIDADOR_TIPO']
    
    elif ( municipio + ' LOGRADOURO ' + row['LOGRADOURO_OCORRENCIA'] ) in df_classif.index:
        return df_classif.loc[municipio + ' LOGRADOURO ' + row['LOGRADOURO_OCORRENCIA'], 'VALIDADOR_TIPO']
    
    elif ( municipio + ' CIDADE ' + row['MUNICIPIO'] ) in df_classif.index:
        return df_classif.loc[municipio + ' CIDADE ' + row['MUNICIPIO'], 'VALIDADOR_TIPO']    

    
    else:
        return 'other'    

    
df_reds['LATITUDE_GEO']  = df_reds.apply(lambda row: preenche_latitude(row), axis=1).astype(np.float64)
df_reds['LONGITUDE_GEO'] = df_reds.apply(lambda row: preenche_longitude(row), axis=1).astype(np.float64)
df_reds['VALIDADOR_TIPO'] = df_reds.apply(lambda row: preenche_valid(row), axis=1)


In [11]:
from shapely.geometry import Point
points = df_reds.apply(lambda row: Point(row.LONGITUDE_GEO, row.LATITUDE_GEO), axis = 1)
df_reds = gpd.GeoDataFrame(df_reds, geometry = points)

In [12]:
df_7rpm = gpd.read_file('geo\\7RPM_Setores_2021v1.shp', epg = "EPSG:4674", encoding='utf-8')
#df_7rpm = gpd.read_file(r'I:\Cloud\Jupyter\diao_final\geo\7RPM_Setores_2021.shp', epg = "EPSG:4674", encoding='utf-8')
df_7rpm.tail(5)

df_7rpm.crs

<Geographic 2D CRS: GEOGCS["GRS 1980(IUGG, 1980)",DATUM["D_unknown",SP ...>
Name: GRS 1980(IUGG, 1980)
Axis Info [ellipsoidal]:
- lon[east]: Longitude (Degree)
- lat[north]: Latitude (Degree)
Area of Use:
- undefined
Datum: D_unknown
- Ellipsoid: GRS80
- Prime Meridian: Greenwich

In [13]:
df_reds = gpd.sjoin(df_reds, df_7rpm[['CIA','SETOR','BAIRRO_GEO','TIPO','geometry']], 'left', op ='within')

<ipython-input-13-7fd9d14c82c8>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: GEOGCS["GRS 1980(IUGG, 1980)",DATUM["D_unknown",SP ...

  df_reds = gpd.sjoin(df_reds, df_7rpm[['CIA','SETOR','BAIRRO_GEO','TIPO','geometry']], 'left', op ='within')


In [14]:
def diao(row):

    conditions = [
        (row['NATUREZA_PRINCIPAL'].str.contains('B01121')),
        (row['NATUREZA_PRINCIPAL'].str.contains('B01148')),
        (row['NATUREZA_PRINCIPAL'].str.contains('C01157')),
        (row['NATUREZA_PRINCIPAL'].str.contains('C01158')),
        (row['NATUREZA_PRINCIPAL'].str.contains('C01159')),
        (row['NATUREZA_PRINCIPAL'].str.contains('D01213')),
        (row['NATUREZA_PRINCIPAL'].str.contains('D01217')),]

    choices = ['B01121', 'B01148', 'C01157', 'C01158', 'C01159', 'D01213', 'D01217']
    df_reds['DIAO'] = np.select(conditions, choices, default='CR')

    return row

df_reds=diao(df_reds)

In [15]:
df_reds = df_reds.loc[
    
    (
    (df_reds['DIAO']!='CR')
    )
]
df_reds

,NUMERO_REDS,TIPO_DE_RELATORIO,DATA/HORA_DE_CRIACAO_DO_REGISTRO,DATA/HORA_DO_FATO,NATUREZA_PRINCIPAL,ENDERECO_DO_FATO,NUMERO_DO_BO,LOGRADOURO_OCORRENCIA,BAIRRO,MUNICIPIO,...,LATITUDE_GEO,LONGITUDE_GEO,VALIDADOR_TIPO,geometry,index_right,CIA,SETOR,BAIRRO_GEO,TIPO,DIAO
337,2021-045144335-001,POLICIAL,18/09/2021 00:50,2021-09-18 00:12:00,ESTUPRO D01213,RUA VIGARIO NICOLAU / - CENTRO - BOM DESPACHO,NaN,VIGARIO NICOLAU,CENTRO,BOM DESPACHO,...,-19.742942,-45.251329,BAIRRO,POINT (-45.25133 -19.74294),332.0,50ª CIA,SUL BOM DESPACHO,JARDIM DOS ANJOS,URBANA,D01213
678,2021-044984750-001,POLICIAL,17/09/2021 09:09,2021-09-17 07:21:00,ROUBO C01157,RUA DEZESSETE / - JARDIM BAVIERA - JUATUBA,NaN,DEZESSETE,JARDIM BAVIERA,JUATUBA,...,0.000000,0.000000,other,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN,NaN,C01157
821,2021-045621605-001,POLICIAL,20/09/2021 22:07,2021-09-20 19:19:00,HOMICIDIO LESAO CORPORAL B01121 B01129,RUA SALVADOR NUNES / - MACHADO - IGARAPE,NaN,SALVADOR NUNES,MACHADO,IGARAPE,...,0.000000,0.000000,other,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN,NaN,B01121
1353,2021-045154800-001,POLICIAL,18/09/2021 02:59,2021-09-18 02:52:00,HOMICIDIO B01121,RODOVIA MG 164 / - ALTO SAO FRANCISCO - MART...,NaN,RODOVIA MG 164,ALTO SAO FRANCISCO,MARTINHO CAMPOS,...,-19.328624,-45.236677,CIDADE,POINT (-45.23668 -19.32862),1932.0,118ª CIA,MARTINHO CAMPOS,BAMBE,URBANA,B01121
1639,2021-045600352-001,POLICIAL,20/09/2021 19:46,2021-09-20 17:50:00,ROUBO C01157,AVENIDA GETULIO VARGAS / - CENTRO - ABAETE,NaN,GETULIO VARGAS,CENTRO,ABAETE,...,-19.144033,-45.451014,CIDADE,POINT (-45.45101 -19.14403),57.0,141ª CIA,ABAETE E FRACOES,SANTA TEREZINHA,URBANA,C01157
1910,2021-044730295-001,POLICIAL,15/09/2021 20:13,2021-09-15 19:56:00,ROUBO C01157,AVENIDA LINCOLN NOGUEIRA / - LINCOLN NOGUEIR...,NaN,LINCOLN NOGUEIRA,LINCOLN NOGUEIRA,NOVA SERRANA,...,-19.854089,-44.976589,BAIRRO,POINT (-44.97659 -19.85409),2187.0,280ª CIA,CONCESSO ELIAS,ANTONIO VENANCIO,URBANA,C01157
1911,2021-044736420-001,POLICIAL,15/09/2021 20:52,2021-09-15 20:12:00,ROUBO C01157,RUA PARA DE MINAS / RUA WASHINGTON LUIZ - CE...,NaN,PARA DE MINAS,CENTRO,NOVA SERRANA,...,-19.875712,-44.983928,BAIRRO,POINT (-44.98393 -19.87571),2030.0,279ª CIA,HIPERCENTRO NSE,BEIJA FLOR,URBANA,C01157
1992,2021-044635601-001,POLICIAL,15/09/2021 11:39,2021-09-15 10:26:00,ROUBO C01157,RODOVIA MG 050 / - VARGINHA - JUATUBA,NaN,RODOVIA MG 050,VARGINHA,JUATUBA,...,0.000000,0.000000,other,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN,NaN,C01157
2136,2021-044924654-001,POLICIAL,16/09/2021 20:48,2021-09-16 20:05:00,ROUBO C01157,RUA DIVISA NOVA / - SALGADO FILHO - BELO HOR...,NaN,DIVISA NOVA,SALGADO FILHO,BELO HORIZONTE,...,0.000000,0.000000,other,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN,NaN,C01157
2216,2021-044783781-001,POLICIAL,16/09/2021 07:03,2021-09-16 06:24:00,ROUBO C01157,RUA LEONARDO AZEVEDO / - SAO GERALDO IL - NO...,NaN,LEONARDO AZEVEDO,SAO GERALDO IL,NOVA SERRANA,...,-19.878186,-44.974373,BAIRRO,POINT (-44.97437 -19.87819),2063.0,279ª CIA,SAO GERALDO,SAO GERALDO IL,URBANA,C01157


In [16]:
df_nat = df_reds['NATUREZA_PRINCIPAL'].str.split(" ", n=20, expand=True)
    
df_reds["NATUREZA"] = df_nat[0]

In [ ]:
# def nat(row):
#     if   row["NATUREZA"] == 'HOMICIDIO':
#         val = 'B01121'
#     elif row["NATUREZA"] == 'ROUBO':
#         val = 'C01157'
#     elif row["NATUREZA"] == 'SEQUESTRO E CARCERE PRIVADO':
#         val = 'B01148'
#     elif row["NATUREZA"] == 'EXTORSAO':
#         val = 'C01158'
#     elif row["NATUREZA"] == 'EXTORSAO MEDIANTE SEQUESTRO':
#         val = 'C01159'
#     elif row["NATUREZA"] == 'ESTUPRO':
#         val = 'D01213'
#     elif row["NATUREZA"] == 'ESTUPRO DE VULNERAVEL':
#         val = 'D01217'
#     else:
        
#         val = 'other'
#     return val

# df_reds['DIAO'] = df_reds.apply(nat, axis=1)


In [17]:
nats = ('B01148', 'B01121', 'C01157', 'C01158', 'C01159','D01213', 'D01217')

is_cv = [
    (df_reds['DIAO'].isin(nats))
        ]

df_reds['TCV'] = np.select(is_cv, [1], default=0)

In [ ]:
df_reds[df_reds['NUMERO_REDS'] == '2021-027721550-001']

In [18]:
conds=[
    #07ºbpm
    df_reds['MUNICIPIO'].isin(['BOM DESPACHO', 'MOEMA']),
    df_reds['MUNICIPIO'].isin(['CORREGO DANTA', 'JAPARAIBA', 'LAGOA DA PRATA', 'LUZ', 'PEDRA DO INDAIA', 'SANTO ANTONIO DO MONTE']),
    df_reds['MUNICIPIO'].isin(['MARTINHO CAMPOS', 'POMPEU']),
    df_reds['MUNICIPIO'].isin(['ABAETE', 'BIQUINHAS', 'CEDRO DO ABAETE', 'DORES DO INDAIA', 'ESTRELA DO INDAIA', 'MORADA NOVA DE MINAS', 'PAINEIRAS', 'QUARTEL GERAL', 'SERRA DA SAUDADE']),
        
    #23ºbpm
    df_reds['MUNICIPIO'].isin(['ITAUNA','ITATIAIUCU']),
    df_reds['SETOR'].isin(['HIPER CENTRO','BOM PASTOR','ALTO GOIAS']),
    df_reds['SETOR'].isin(['PLANALTO','SAO JOSE','CLAUDIO']),
    df_reds['SETOR'].isin(['NITEROI','PORTO VELHO','C.CAJURU/S.G.PARA']),
    
    #60ºbpm
    df_reds['SETOR'].isin(['PERDIGAO/ARAUJOS', 'SUPER CENTRO', 'ROMEU DUARTE', 'SAO GERALDO']),
    df_reds['SETOR'].isin(['CONCEICAO/PITANGUI', 'LEANDRO FERREIRA', 'CONCESSO ELIAS', 'SANTA SARA', 'ZONA RURAL']),
        
    #63ºbpm
    df_reds['MUNICIPIO'].isin(['ARCOS', 'BAMBUI', 'IGUATAMA', 'MEDEIROS', 'PAINS', 'TAPIRAI']),
    df_reds['MUNICIPIO'].isin(['CAMACHO', 'CORREGO FUNDO', 'FORMIGA', 'ITAPECERICA', 'PIMENTA', 'SAO SEBASTIAO DO OESTE']),
        
    #19ª cia pm ind
    df_reds['MUNICIPIO'].isin(['IGARATINGA', 'MARAVILHAS', 'ONCA DO PITANGUI', 'PAPAGAIOS', 'PARA DE MINAS', 'PEQUI', 'SAO JOSE DA VARGINHA']),    

    ]
res1 = ['50ª CIA','107ª CIA','118ª CIA','141ª CIA']
res2 = ['51ª CIA','53ª CIA','139ª CIA','142ª CIA']
res3 = ['279ª CIA','280ª CIA']
res4 = ['241ª CIA','290ª CIA']
res5 = ['19ª CIA IND']
resf = res1 + res2 + res3 + res4 +res5
df_reds['CIA'] = np.select(conds,resf,default='other')

In [19]:
conds=[
    df_reds['MUNICIPIO'].isin(['ABAETE','BIQUINHAS','BOM DESPACHO','CEDRO DO ABAETE','CORREGO DANTA',
		'DORES DO INDAIA','ESTRELA DO INDAIA','JAPARAIBA','LAGOA DA PRATA','LUZ','MARTINHO CAMPOS','MOEMA',
		'MORADA NOVA DE MINAS','PAINEIRAS','PEDRA DO INDAIA','POMPEU','QUARTEL GERAL','SANTO ANTONIO DO MONTE',
		'SERRA DA SAUDADE']),
    df_reds['MUNICIPIO'].isin(['CARMO DO CAJURU','CLAUDIO','DIVINOPOLIS','ITATIAIUCU','ITAUNA','SAO GONCALO DO PARA']),
    df_reds['MUNICIPIO'].isin(['ARAUJOS','CONCEICAO DO PARA','LEANDRO FERREIRA','NOVA SERRANA','PERDIGAO','PITANGUI']),
    df_reds['MUNICIPIO'].isin(['ARCOS','BAMBUI','CAMACHO','CORREGO FUNDO','FORMIGA','IGUATAMA','ITAPECERICA','MEDEIROS','PAINS','PIMENTA','SAO SEBASTIAO DO OESTE','TAPIRAI']),
    df_reds['MUNICIPIO'].isin(['IGARATINGA','MARAVILHAS','ONCA DO PITANGUI','PAPAGAIOS','PARA DE MINAS','PEQUI','SAO JOSE DA VARGINHA']),
]
res=['07º BPM','23º BPM','60º BPM','63º BPM','19ª CIA PM IND']
df_reds['UEOP'] = np.select(conds,res,default='other')

In [20]:
conds=[
    df_reds['MUNICIPIO'].isin(['ABAETE','BIQUINHAS','BOM DESPACHO','CEDRO DO ABAETE','CORREGO DANTA',
     'DORES DO INDAIA','ESTRELA DO INDAIA','JAPARAIBA','LAGOA DA PRATA','LUZ','MARTINHO CAMPOS','MOEMA',
    'MORADA NOVA DE MINAS','PAINEIRAS','PEDRA DO INDAIA','POMPEU','QUARTEL GERAL','SANTO ANTONIO DO MONTE',
    'SERRA DA SAUDADE',#7º BPM
    'CARMO DO CAJURU','CLAUDIO','DIVINOPOLIS','ITATIAIUCU','ITAUNA','SAO GONCALO DO PARA',#23º BPM
    'ARAUJOS','CONCEICAO DO PARA','LEANDRO FERREIRA','NOVA SERRANA','PERDIGAO','PITANGUI',#60º BPM
    'ARCOS','BAMBUI','CAMACHO','CORREGO FUNDO','FORMIGA','IGUATAMA','ITAPECERICA','MEDEIROS','PAINS','PIMENTA','SAO SEBASTIAO DO OESTE','TAPIRAI',#63º BPM
    'IGARATINGA','MARAVILHAS','ONCA DO PITANGUI','PAPAGAIOS','PARA DE MINAS','PEQUI','SAO JOSE DA VARGINHA'])#19ª cia ind
]
res=['07ª RPM']
df_reds['UDI'] = np.select(conds,res,default='other')

In [ ]:
#df_reds[df_reds['UDI'] == '07ª RPM']

In [ ]:
# Fator impeditivo pois não é possivel  saber se é consumado ou tentado
# df_reds_hc = df_reds.loc[
    
#     (
#     (df_reds['TIPO_DE_RELATORIO']=='POLICIAL') &
#         (df_reds['UDI']=='07ª RPM') &
#         (df_reds['DIAO']== 'B01121')
#     )
# ]
# df_reds_hc

In [22]:
df_reds = df_reds.loc[
    
    (
    (df_reds['TIPO_DE_RELATORIO']=='POLICIAL') &
        (df_reds['UDI']=='07ª RPM') &
        (df_reds['TCV']== 1)
    )
]


In [ ]:
# import os
# def path_remove():
    
#     if os.path.exists(r'I:\Cloud\SQLite\10 - Source\transacional'):#home
#         dir_save  = r"I:\Cloud\SQLite\10 - Source\transacional\relatorio.csv"
#     elif os.path.exists(r"D:\Cloud\SQLite\10 - Source\transacional"):#office
#         dir_save  = r"D:\Cloud\SQLite\10 - Source\transacional\relatorio.csv"
#     else:
#         dir_save  = "Arquivo não encontrado!"
#     return dir_save    
# path_remove()
# os.remove(path_remove())

In [23]:
df_reds = df_reds[['NUMERO_REDS', 'DATA_FATO', 'HORARIO_FATO', 'DATA_ULTIMA_ATUALIZACAO',
                   'DIAO', 'TENTADO_CONSUMADO_NAT_PRINCIPAL', 'ALVO', 'CAUSA_PRESUMIDA',
                   'TCV', 'LOGRADOURO_OCORRENCIA', 'BAIRRO', 'MUNICIPIO', 'FRACAO', 'CIA', 
                   'UEOP', 'SETOR', 'LATITUDE', 'LONGITUDE']]

In [24]:
def path_save():
    import os
    if os.path.exists(r'I:\Cloud\SQLite\10 - Source\transacional'):#home
        dir_save  = r"I:\Cloud\SQLite\10 - Source\transacional\transacional_tratado.xlsx"
    elif os.path.exists(r"D:\Cloud\SQLite\10 - Source\transacional"):#office
        dir_save  = r"D:\Cloud\SQLite\10 - Source\transacional\tbl_transacional_tratado.xlsx"
    else:
        dir_save  = "Arquivo não encontrado!"
    return dir_save    
path_save()


'D:\\Cloud\\SQLite\\10 - Source\\transacional\\tbl_transacional_tratado.xlsx'

In [ ]:
df_reds.dtypes

In [ ]:
df_reds['NUMERO_REDS'].count()

In [25]:
def path_db():
    import os
    if os.path.isfile(r'I:\Cloud\SQLite\10 - Source\bd_main\gdo.db'):#home
        dir_save  = r"I:\Cloud\SQLite\10 - Source\bd_main\gdo.db"
    else:
        os.path.isfile(r'D:\Cloud\SQLite\10 - Source\bd_main\gdo.db')#office
        dir_save  = r"D:\Cloud\SQLite\10 - Source\bd_main\gdo.db"
    return "sqlite:///" + dir_save    
path_db()

'sqlite:///D:\\Cloud\\SQLite\\10 - Source\\bd_main\\gdo.db'

In [26]:
df_ocorrencias = pd.read_sql_table('tbl_crimes', path_db(),)

In [ ]:
df_reds #NUMERO_REDS

In [27]:
df_ocorrencias.loc[
    
    (
    (df_ocorrencias['DATA_FATO'].dt.year == 2021) &
        (df_ocorrencias['TCV']== 1)
    )
].count()

NUMERO_REDS                             1337
ORGAO_UNIDADE_REGISTRO                  1335
UNIDADE_AREA_MILITAR                    1335
UNIDADE_RELATO_NIVEL_6                  1335
UNID_DIGITACAO_NIVEL_6                  1335
UNID_REGISTRO_NIVEL_6                   1335
CODIGO_SUBCLASSE_NAT_PRINCIPAL          1335
TENTADO_CONSUMADO_NAT_PRINCIPAL         1337
CODIGO_SUBCLASSE_NAT_SEC1               1335
TENTADO_CONSUMADO_NAT_SEC1              1335
CODIGO_SUBCLASSE_NAT_SEC2               1335
TENTADO_CONSUMADO_NAT_SEC2              1335
CODIGO_SUBCLASSE_NAT_SEC3               1335
TENTADO_CONSUMADO_NAT_SEC3              1335
DESC_LONGA_SUBGRUPO_COMPLEMENTO_NAT     1335
DESCRICAO_MEIO_UTILIZADO                1335
CAUSA_PRESUMIDA                         1337
DATA_COMUNICACAO_FATO                      0
HORARIO_COMUNICACAO_FATO                1281
DATA_FATO                               1337
HORARIO_FATO                            1337
DATA_ULTIMA_ATUALIZACAO                 1337
LOGRADOURO

In [28]:
df_reds = df_reds[~df_reds['NUMERO_REDS'].isin(df_ocorrencias['NUMERO_REDS'])]
df_reds




,NUMERO_REDS,DATA_FATO,HORARIO_FATO,DATA_ULTIMA_ATUALIZACAO,DIAO,TENTADO_CONSUMADO_NAT_PRINCIPAL,ALVO,CAUSA_PRESUMIDA,TCV,LOGRADOURO_OCORRENCIA,BAIRRO,MUNICIPIO,FRACAO,CIA,UEOP,SETOR,LATITUDE,LONGITUDE


In [29]:
df_reds.to_sql('tbl_crimes', path_db(), if_exists='append', index=False)

In [30]:
with pd.ExcelWriter(path_save()) as save:
    df_reds.to_excel(save, sheet_name='tbl_transacional', index = False)
    

In [ ]:
# save = r"C:\Users\Geo\Desktop\reds_transacional.xlsx"
# df_reds.to_excel(save, sheet_name='reds', index = False)

In [ ]:
# df_reds['data'] = df_reds['Data/Hora do Fato'].str.split(pat=" ")


# df_reds['data']

In [ ]:
# df_reds['Natureza Principal'][
#     ( df_reds['Natureza Principal'].str.contains("C01157", case = False,  na = False, regex = True) )
# ]

In [ ]:
# df_reds['Natureza Principal'][df_reds['Natureza Principal'].str.contains("C01157", case = False,  na = False, regex = True)]

In [ ]:



# if df_reds['Natureza Principal'][df_reds['Natureza Principal'].str.contains("C01157", case = False,  na = False, regex = True)]:
#     val = 'roubos'
# else:
#     val = 'outro'
# return val

In [ ]:
# def nat(row):
#     if   row['Natureza Principal'].str.contains("C01157", case = False,  na = False, regex = True) == True:
#         val = 'C01157'
#     elif row['Natureza Principal'].str.contains("B01121", case = False,  na = False, regex = True) == True:
#         val = 'B01148'
   
#     else:
#         val = row['Natureza Principal']
#         #val = None
#     return val

# df_reds['DIAO'] = df_reds.apply(nat, axis=1)